load the pre-processed dataset

In [ ]:
df = pd.read_csv("df.csv")

In [ ]:
import openai
from bertopic.representation import OpenAI
from bertopic import BERTopic

# Create your representation model
client = openai.OpenAI(api_key="api_key")
# representation_model = OpenAI(client)


In [ ]:
# System prompt describes information given to all conversations
system_prompt = """
You are a helpful, respectful and honest assistant for labeling topics.
"""

# Example prompt demonstrating the output we are looking for
example_prompt = """
I have a topic that contains the following documents:
- Traditional diets in most cultures were primarily plant-based with a little meat on top, but with the rise of industrial style meat production and factory farming, meat has become a staple food.
- Meat, but especially beef, is the word food in terms of emissions.
- Eating meat doesn't make you a bad person, not eating meat doesn't make you a good one.

The topic is described by the following keywords: 'meat, beef, eat, eating, emissions, steak, food, health, processed, chicken'. And a title 'The Evolution of Diets: From Plant-Based to Meat-Centric'.

Based on the information about the topic above, please create a short label of this topic and title. Make sure you to only return the label and title, nothing more.

"""


main_prompt = """
I have a topic that contains the following documents:
[DOCUMENTS]
The topic is described by the following keywords: [KEYWORDS]

Based on the information above, extract a short topic label in the following format:
topic: <topic label>
"""


prompt = system_prompt + example_prompt + main_prompt


In [ ]:
seed_words = ["Language","Dataset","Corpus Size","Computational","Machine Translation","Tokenization","Costs"
  "Interpretability", "Morphology","Semantic","Memory","Skewed distributions","Biased Distributions","Small size",
                "Generalizability","Bias","Hyperparameter","Hardware","Real world","Robustness", "Noisy", "Time",
                "Annotations", "Evaluation", "Diversity", "Segmentation", "Metrics"]

In [ ]:
# context window 128,000 tokens, returns maximum 4,096 tokens
from sklearn.metrics import silhouette_score
from bertopic import BERTopic
from umap import UMAP
import numpy as np
from hdbscan import HDBSCAN
from bertopic.vectorizers import ClassTfidfTransformer
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
import openai
import tiktoken
from bertopic.representation import OpenAI
from bertopic import BERTopic
# from bertopic.representation import KeyBERTInspired

zeroshot_topic_list = seed_words
silhouette_score_list = []
sentence_model = SentenceTransformer("all-MiniLM-L6-v2")
# cluster_model = KMeans(31)
embeddings = sentence_model.encode(df['Text'], show_progress_bar=False)
# Tokenizer
tokenizer= tiktoken.encoding_for_model("gpt-3.5-turbo")
# others: gpt-4-turbo-preview	, gpt-4, 	gpt-4-0613	, gpt-4-32k	, gpt-4-32k-0613

# umap_model = UMAP(n_neighbors=13, n_components=15,
#                     min_dist=0.0, metric='cosine',random_state=42) # If you want to reproduce the results, at the expense of performance, you can set a random_state in UMAP to prevent any stochastic behavior:

vectorizer_model = CountVectorizer(stop_words="english")

client = openai.OpenAI(api_key="api_key")

representation_model = OpenAI(
    client,
    model="gpt-3.5-turbo",
    delay_in_seconds=2,
    chat=True,
    nr_docs=4,
    doc_length=100,
    tokenizer=tokenizer,
    prompt = prompt
)

# Use the representation model in BERTopic on top of the default pipeline
# topic_model = BERTopic(representation_model=representation_model,umap_model=umap_model)

topic_model = BERTopic(
    embedding_model="thenlper/gte-small",
    min_topic_size=10,
    zeroshot_topic_list=zeroshot_topic_list,
    zeroshot_min_similarity=.75,
    representation_model=representation_model,
    vectorizer_model=vectorizer_model
)

topics, probs = topic_model.fit_transform(df['Text'], embeddings)

# Generate `X` and `labels` only for non-outlier topics (as they are technically not clusters)
umap_embeddings = topic_model.umap_model.transform(embeddings)
indices = [index for index, topic in enumerate(topics) if topic != -1]
X = umap_embeddings[np.array(indices)]
labels = [topic for index, topic in enumerate(topics) if topic != -1]

  # Calculate silhouette score
silhouette_score(X, labels)


In [ ]:
most_representative_words_llm = topic_model.get_topic_info()
most_representative_words_llm

,Topic,Count,Name,Representation,Representative_Docs
0,-1,978,-1_Evaluation of Methods for Text Classificati...,[Evaluation of Methods for Text Classification...,[Model sizes and comparability as we have poin...
1,0,230,0_Challenges and Approaches in Multilingual La...,[Challenges and Approaches in Multilingual Lan...,[Even the premise of parsing questions to wiki...
2,1,113,1_Evaluation of Dialogue Systems and Model Lim...,[Evaluation of Dialogue Systems and Model Limi...,"[In this paper, we present a comprehensive fra..."
3,2,83,2_Limitations of Multi-modal Approaches in Sce...,[Limitations of Multi-modal Approaches in Scen...,[Limitations of data collection our proposed d...
4,3,59,3_Long-document summarization models and techn...,[Long-document summarization models and techni...,[We propose a set prediction network for the e...
5,4,55,4_Reasoning in Large Language Models with Expl...,[Reasoning in Large Language Models with Expla...,"[Concepts in this work, we extract the concept..."
6,5,52,5_Handling label noise and entity classificati...,[Handling label noise and entity classificatio...,[Safer framework is designed for handling bert...
7,6,47,6_Limitations of clinical study on cancer prog...,[Limitations of clinical study on cancer progn...,[This study is based on a single clinical coho...
8,7,47,7_Defense against Adversarial Attacks in NLP A...,[Defense against Adversarial Attacks in NLP Ap...,[Auxiliary attack and supports: votetrans with...
9,8,45,8_Question Generation Models and Evaluation wi...,[Question Generation Models and Evaluation wit...,[Limitation of this study is that we used only...


In [ ]:
for i in range(len(most_representative_words_llm)):
  print(most_representative_words_llm["Name"][i])

-1_Evaluation of Methods for Text Classification and Relation Extraction
0_Challenges and Approaches in Multilingual Language Translation and Model Performance
1_Evaluation of Dialogue Systems and Model Limitations
2_Limitations of Multi-modal Approaches in Scene Understanding and Data Collection
3_Long-document summarization models and techniques
4_Reasoning in Large Language Models with Explanations and GPT Models
5_Handling label noise and entity classification challenges in NER with domain-specific data and self-training methods
6_Limitations of clinical study on cancer prognostic prediction and evaluation of generated clinical notes
7_Defense against Adversarial Attacks in NLP Applications
8_Question Generation Models and Evaluation with Knowledge Graph Augmentation
9_Limitations in Transformer Model Studies
10_Limitations of NLP Research Methodology
11_Knowledge Graphs and Retrieval Methods
12_Limitations and Considerations in Deploying Compressed GPU Models for Language Understa

keyBERT

In [ ]:
!pip install keybert

In [ ]:
# Convert each list in the 'Representation' column to a string
df['Representation'] = df['Representation'].apply(lambda x: ''.join(map(str, x)))

In [ ]:
from keybert import KeyBERT

kw_model = KeyBERT()
doc_embeddings, word_embeddings = kw_model.extract_embeddings(df["Representation"])

In [ ]:
keywords = kw_model.extract_keywords(df["Representation"], doc_embeddings=doc_embeddings, word_embeddings=word_embeddings)
keywords

[[('annotators', 0.4626),
  ('annotation', 0.4237),
  ('adversarial', 0.4186),
  ('chatbot', 0.4052),
  ('text', 0.3639)],
 [('multilingual', 0.5756),
  ('lingual', 0.4836),
  ('languages', 0.3962),
  ('translation', 0.3692),
  ('language', 0.3547)],
 [('dialogue', 0.5715),
  ('evaluation', 0.3413),
  ('assessment', 0.2973),
  ('initiative', 0.2954),
  ('texts', 0.2631)],
 [('multimodal', 0.519),
  ('visual', 0.3682),
  ('encoding', 0.341),
  ('datasets', 0.3375),
  ('text', 0.3275)],
 [('deductive', 0.4963),
  ('reasoning', 0.4603),
  ('learners', 0.3994),
  ('explanations', 0.3828),
  ('learning', 0.3662)],
 [('2wikihopmultiqa', 0.3491),
  ('qa', 0.3367),
  ('hop', 0.3256),
  ('answering', 0.3249),
  ('qag', 0.3201)],
 [('summarization', 0.5498),
  ('dialogue', 0.3914),
  ('lengthy', 0.3882),
  ('text', 0.3453),
  ('document', 0.2943)],
 [('adversarial', 0.5353),
  ('nlp', 0.4385),
  ('bert', 0.3726),
  ('attacks', 0.3321),
  ('overfitting', 0.3249)],
 [('clinical', 0.4105),
  ('nlp'

In [ ]:
import nltk
import gensim
import gensim.corpora as corpora
import pandas as pd
import numpy as np
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from bertopic.vectorizers import ClassTfidfTransformer
from gensim.models.coherencemodel import CoherenceModel
import pandas as pd
import re

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_20newsgroups

from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary


words_only = [[word for word, _ in sublist] for sublist in keywords]
print(words_only)
# convert list to string
df['Representation'] = df['Representation'].apply(lambda x: ''.join(x))
# Tokenize the strings to create a list of lists
list_of_tokens = df['Representation'].apply(lambda x: x.split()).tolist()

print(list_of_tokens)
word2id = Dictionary( list_of_tokens )

# Coherence model
cm = CoherenceModel(topics= words_only,
                    texts=list_of_tokens,
                    coherence='c_v',
                    dictionary=word2id)

coherence_per_topic = cm.get_coherence_per_topic()
print(coherence_per_topic)

import statistics
statistics.mean(coherence_per_topic)

# visualization

In [ ]:
topic_model.visualize_topics()


In [ ]:
topic_model.visualize_heatmap()


In [ ]:
from umap import UMAP

# Run the visualization with the original embeddings
topic_model.visualize_documents(df['Text'], embeddings=embeddings)

# Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings)
topic_model.visualize_documents(df['Text'], reduced_embeddings=reduced_embeddings)

In [ ]:
from bertopic import BERTopic

# topic_model = BERTopic()
# topics, _ = topic_model.fit_transform(docs)
topic_distr, _ = topic_model.approximate_distribution(df['Text'], min_similarity=0)


In [ ]:
# don't use embedding_model = sentence model in 'BERTopic'
topic_model = BERTopic(representation_model=representation_model,vectorizer_model=vectorizer_model,
                         seed_topic_list = seed_words, verbose=True, calculate_probabilities=True)

topics, probs = topic_model.fit_transform(df['Text'], embeddings)

2024-03-17 09:46:34,463 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-03-17 09:47:00,728 - BERTopic - Dimensionality - Completed ✓
2024-03-17 09:47:00,730 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-03-17 09:47:00,975 - BERTopic - Cluster - Completed ✓
2024-03-17 09:47:00,981 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 30/30 [01:23<00:00,  2.77s/it]
2024-03-17 09:48:25,032 - BERTopic - Representation - Completed ✓


In [ ]:
# To visualize the probabilities of topic assignment
topic_model.visualize_distribution(probs[0])

# To visualize the topic distributions in a document
topic_model.visualize_distribution(topic_distr[0])
